<div class="alert alert-block alert-info">

<b>Note:</b> This is a work in progress notebook!   

</div>

*Also, check my "Fast solution. Coleridge Initiative: Baseline" which i used to generate the first version of the wordmodel dataset*

*(https://www.kaggle.com/nikitakudriashov/fast-solution-coleridge-initiative-baseline)*

# Coleridge Initiative solution

### Named Entity Recognition task (BIO labeling + Glove Embadding + biLSTM + CDF layer)
----


#### 👋 Hi, I am new to NLP and espessially to Named Entity Recognition.
 
#### 🎓 Actually, I took this task just to get more envolved into this sphere and to sort out my knowledge.
 
#### 💸 I hope that it will be useful for you.
 
#### 🧠 I will be glad to hear your advices and best practices examples
 
#### 👍 And ofcourse I'll be happy to geet your upvotes, enjoy

In [ ]:
from tqdm.notebook import tqdm
from functools import partial
import pandas as pd
import numpy as np
import gc
import re
import json
import nltk

from nltk.stem import WordNetLemmatizer
from nltk.tokenize import sent_tokenize, word_tokenize
from sklearn.model_selection import train_test_split

pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', None)

In [ ]:
def get_raw_txt(_id:str, path="train"):
    """
    Gets row publication text by its _id
    :param _id: publication string id 
    :param path: root directory to the publication json files - train/test
    :return: raw string of text
    """
    _d = json.loads(open(f"{PATH}/{path}/{_id}.json").read())
    return " ".join([i["text"] for i in _d])

def clean_text(txt:str):
    """
    Code snippet from the main competition page
    """
    return re.sub('[^A-Za-z0-9]+', ' ', str(txt).lower()).strip()

def get_lbl_positions(label:str, text:str):
    """
    Method provides the position of the 1st and the last word of the label in the given text
    :return: None if no match or _f,_l - for the first and the last word positions
    """
    _f = re.search(label+" ",text)
    if _f:
        _f = len(word_tokenize(text[:_f.start()]))
        _l = _f + len(word_tokenize(label))
        return int(_f), int(_l)
        
def encode_bio_by_position(text:list, positions:list, padding:int):
    """
    Returns bio encoded string by its features
    :param positions: list of tuples
    """
    bio = np.zeros((padding,),dtype=np.int8)
    for _f, _l in positions:
        bio[_f] = 2 #B
        bio[_f+1:_l] = 1 #I
    return bio

## 1. Data Preprocessing

### Train set transformation
On the previouse steps i have generated the wordset from the matches with the text in the train_df. Now we need to relable it with the aim of:
* It is not fully labeled (not all matches are taken into account)
* I will group it in a form where Id is a primar key, so our dataset length equels to the number of publications
* Our train dataset will be putted into the same form as the test one wil be (Id & PredictionString)

In [ ]:
PATH = "../input/coleridgeinitiative-show-us-the-data/"
tqdm_params = dict(bar_format='{desc}{bar} [ {n} / {total} (remaining: {remaining}) ]', colour="darkgreen")
wordmodel = pd.read_csv("../input/wordmodel/wordmodel.csv").values.squeeze()
wordmodel = sorted(wordmodel, key=len)

tqdm.pandas(desc="Text download from JSON files status :",**tqdm_params)
df_train = pd.DataFrame(np.unique(pd.read_csv(f"{PATH}train.csv")[["Id"]]), columns = ["Id"])
df_train['text'] = df_train['Id'].progress_apply(partial(get_raw_txt))

tqdm.pandas(desc="Text cleaning with clean_text function :",**tqdm_params)
df_train['text'] = df_train['text'].progress_apply(clean_text)

df_train['PredictionString'] = ''
desc="Grouping & relabeling by the wordmodel :"
for i in tqdm(df_train.index, desc=desc ,**tqdm_params):
    for ws in wordmodel:
        if ws in df_train.loc[i]["text"]:
            df_train.loc[i]['PredictionString'] += ws.strip() + '|'
    df_train.loc[i]['PredictionString'] = df_train.loc[i]['PredictionString'][:-1]

df_train = df_train.drop("text",axis=1)

In [ ]:
df_train.sample(10)

### BIO Labeling
I did the BIO labeling with the self-written code, to have more control of it. It looks like not be the optimal solution, may work a bit long. There are some bottlenecks for the labeling:
* Some sentances have a really huge length after the tokenization. To handle such of them, we simply prune all the values on the positions after the maxlen.
* When prunning the values, we need to check if our lable exist in non-pruned values or not.
* We need to collect the BIO-labeled sequences into one sequence, when multipal matches are in text

In [ ]:
x,y=[],[]
maxlen = 100
desc="X & Y extraction + BIO labeling :"
counter, too_long = 0 , []
for _id in tqdm(df_train.index, desc=desc, **tqdm_params):
    # Getting list of cleaned sentances from text by Id    
    _txt = [clean_text(_i) for _i in sent_tokenize(get_raw_txt(df_train.loc[_id,"Id"]))]
    # Getting list of labels
    _labels = df_train.loc[_id,"PredictionString"].split('|')
    for _sent in _txt:
        _positions = []
        for _label in _labels:
            _pos = get_lbl_positions(_label,_sent)
            # Non-pruned labels check
            if _pos:
                if _pos[1] <= maxlen:
                    _positions.append(_pos)
        if _positions:
            x.append(word_tokenize(_sent)[:maxlen])
            y.append(encode_bio_by_position(_sent,_positions,maxlen))

In [ ]:
print(" ".join(x[0]))
print(" ".join([str(_i) for _i in y[0]]))

### GloVe embedding
Again,i decided to apply Glove embadding manually, without putting it into the Keras layer, to save the model train performance 
I took the Stanford's GloVe 100d word embeddings

In [ ]:
glove = open('../input/glove6b100dtxt/glove.6B.100d.txt', encoding="utf8")
glove_dict = {}

desc="Filling up the GloVe dict :"
for _line in tqdm(glove, desc=desc, **tqdm_params):
    records = _line.split()
    glove_dict[records[0]] = [float(i) for i in records[1:]]

In [ ]:
maxlen = max([len(i) for i in x])
wnl = WordNetLemmatizer()
zero_vector = [0]*100

desc="Embedding with the GloVe vectors :"
for _i in tqdm(range(len(x)), desc=desc, **tqdm_params):
    for _j in range(len(x[_i])):
        try:
            x[_i][_j] = glove_dict[wnl.lemmatize(x[_i][_j])]
        except KeyError:
            x[_i][_j] = zero_vector

desc="Padding with the zero vaectors :"
for _i in tqdm(range(len(x)), desc=desc, **tqdm_params):
    _pad = maxlen - len(x[_i])
    for _j in range(_pad):
        x[_i].append(zero_vector)

In [ ]:
x_tr,x_val,y_tr,y_val = train_test_split(np.array(x),np.array(y),test_size=0.3, shuffle=True, random_state=0)

In [ ]:
del x,y,wordmodel, ws, train_test_split, too_long, df_train, zero_vector
gc.collect()

In [ ]:
%who

In [ ]:
del WordNetLemmatizer, clean_text, counter, desc, encode_bio_by_position, get_lbl_positions, get_raw_txt, glove, glove_dict
#del x,y,wordmodel, ws, train_test_split, too_long, df_train, zero_vector
gc.collect()

## 2. Model Fitting

In [ ]:
!pip3 install ../input/keras-crf-whl-files/seqeval-1.2.2-py3-none-any.whl
!pip3 install ../input/keras-crf-whl-files/keras_crf-0.2.0-py3-none-any.whl

In [ ]:
from keras_crf import CRFModel
from keras import Input, Model
from keras.optimizers import Adam
from keras.layers import Bidirectional,LSTM, Dense,TimeDistributed

from keras.metrics import Recall

In [ ]:
inputs = Input(shape=(maxlen,100))
outputs = Bidirectional(LSTM(units=maxlen, return_sequences=True,recurrent_dropout=0.1))(inputs)
outputs = TimeDistributed(Dense(maxlen, activation="relu"))(outputs) 
model = Model(inputs, outputs)
model = CRFModel(model,3)

model.compile(Adam(lr=0.0005,beta_1=0.9,beta_2=0.999,epsilon=1e-07,),metrics=['acc'])
model.fit(x_tr, y_tr, epochs=100, batch_size=2000, validation_data=(x_val,y_val))

In [ ]:
model.save("model")
!zip -r model.zip model

## 3. Prediction on the test set

In [ ]:
del x_tr,x_val,y_tr,y_val
gc.collect()

In [ ]:
from tqdm.notebook import tqdm
import json
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import sent_tokenize, word_tokenize
from sklearn.model_selection import train_test_split

In [ ]:
glove = open('../input/glove6b100dtxt/glove.6B.100d.txt', encoding="utf8")
glove_dict = {}

desc="Filling up the GloVe dict :"
for _line in tqdm(glove, desc=desc, **tqdm_params):
    records = _line.split()
    glove_dict[records[0]] = [float(i) for i in records[1:]]

In [ ]:
PATH = "../input/coleridgeinitiative-show-us-the-data/"
tqdm_params = dict(bar_format='{desc}{bar} [ {n} / {total} (remaining: {remaining}) ]', colour="darkgreen")
wordmodel = pd.read_csv("../input/wordmodel/wordmodel.csv").values.squeeze()
wordmodel = sorted(wordmodel, key=len)

tqdm.pandas(desc="Text download from JSON files status :",**tqdm_params)
deploy = pd.read_csv(f"{PATH}sample_submission.csv")

deploy.index = deploy['Id']

Prediction procedure should be optimised - just the first brief version

In [ ]:
maxlen = 100
zero_vector = [0]*100
wnl = WordNetLemmatizer()
debinarize  = lambda arr: {"100":"0","010":"1","001":"2"}["".join([str(i) for i in arr])]

for _id in deploy.index:
    print(_id)
    result = []
    _txt = [clean_text(_i) for _i in sent_tokenize(get_raw_txt(deploy.loc[_id,"Id"]))]
    for _sent in _txt:
        _sent = word_tokenize(_sent[:maxlen])
        _row_sent = _sent.copy()
        
        #Glove Embadding
        for _i in range(len(_sent)):
            try:
                _sent[_i] = glove_dict[wnl.lemmatize(_sent[_i])]
            except KeyError:
                _sent[_i] = zero_vector

        #Padding with the zero vaectors
        _pad = maxlen - len(_sent)
        for _j in range(_pad):
            _sent.append(zero_vector)
        
        prediction = model.predict(np.array([_sent])).astype(int)
        prediction = "".join([debinarize(i) for i in prediction[0]])
        prediction = re.finditer(r"(21*)",prediction)
        if prediction:
            for i in prediction:
                # I filter one-words for now, need to be emprooved
                if i.span()[1]- i.span()[0]>1:
                    result.append(" ".join(_row_sent[i.span()[0]:i.span()[1]]))
    print("|".join(set(result)))
    print("---")
    deploy.loc[_id,"PredictionString"]="|".join(set(result))


In [ ]:
deploy.to_csv("deploy.csv", index=False)